## Import and install

In [9]:
#!conda install -c conda-forge beautifulsoup4 --yes 
from bs4 import BeautifulSoup

In [2]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner

## Download data

In [5]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
results = requests.get(url)

soup = BeautifulSoup(results.text, "html.parser")

<Response [200]>

## Process data

In [86]:
soup.table.tbody

# Create Dataframe

row = []
for tr in soup.table.tbody.find_all('tr'):
    td = tr.find_all('td')
    if len(td)==0:
        th = tr.find_all('th')
        col=[str(th[0].string).rstrip(),str(th[1].string).rstrip(),str(th[2].string).rstrip()]
    else:
        #if (td[2].string.find("Not assigned") > 0):
        row.append([str(td[0].string).rstrip(),str(td[1].string).rstrip(),str(td[2].string).rstrip()]) 

#col = ['Postcode','Borough','Neighbourhood']
df_pc = pd.DataFrame(row,columns=col)

# Remove NA

df_pc = df_pc[df_pc[col[1]] != 'Not assigned']

# asign Neighbourhood by Borough


# Group Postcode

#df_pc = df_pc.groupby([col[0],col[1]]).agg({col[2]: lambda x: x.nunique()})
df_pc = df_pc.groupby([col[0],col[1]])[col[2]].agg(lambda x: ','.join(x)).reset_index(name ='col[2]')
df_pc.columns = col

df_pc[col[2]].replace('None','',inplace=True)
df_pc[col[2]].replace('None,None','',inplace=True)


df_pc[col[2]] = np.where(df_pc[col[2]] == 'Not assigned', df_pc[col[1]] , df_pc[col[2]] )
df_pc[col[2]] = np.where(df_pc[col[2]] == '', df_pc[col[1]] , df_pc[col[2]] )
df_pc[col[2]] = np.where(df_pc[col[2]] == 'None,None,None', df_pc[col[1]] , df_pc[col[2]] )

df_pc




,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,Scarborough
1,M1C,Scarborough,Scarborough
2,M1E,Scarborough,"Guildwood,None,None"
3,M1G,Scarborough,Scarborough
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough
6,M1K,Scarborough,"East Birchmount Park,None,None"
7,M1L,Scarborough,Scarborough
8,M1M,Scarborough,"None,None,Scarborough Village West"
9,M1N,Scarborough,"None,Cliffside West"


In [87]:
df_pc.shape

(103, 3)

In [ ]:
#!conda install -c conda-forge geocoder --yes 
import geocoder # import geocoder

df_pc['Latitude'],df_pc['Longitude'] =  df_pc.apply(GetLocation, axis = 1) 

# initialize your variable to None
def GetLocation(x):
    lat_lng_coords = None
    # loop until you get the coordinates
    #while(lat_lng_coords is None):
    g = geocoder.google('{}, {}'.format(x.Postcode,x.Neighbourhood))
    lat_lng_coords = g.latlng

    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    return latitude,longitude
df_pc